In [1]:
import pandas as pd
import plotly.express as px

In [2]:
group_a = pd.read_csv(r'F:\SF-DS-practice\Block-3\data\4.6-4.9\ab_test-redesign_sample_a.zip')
group_b = pd.read_csv(r'F:\SF-DS-practice\Block-3\data\4.6-4.9\ab_test-redesign_sample_b.zip')
display(
    group_a.head(),
    group_b.head()
)

,date,cid,transactions,revenue,group
0,2020-11-06,1.001648e+09,0,0,A
1,2020-11-06,1.001936e+09,0,0,A
2,2020-11-06,1.003214e+08,0,0,A
3,2020-11-06,1.004393e+09,0,0,A
4,2020-11-06,1.006382e+09,0,0,A


,date,cid,transactions,revenue,group
0,2020-11-06,1.000135e+09,0,0,B
1,2020-11-06,1.001648e+09,0,0,B
2,2020-11-06,1.003214e+08,0,0,B
3,2020-11-06,1.003652e+09,0,0,B
4,2020-11-06,1.003833e+09,0,0,B


In [3]:
full_data = pd.concat([group_a, group_b], axis=0)
full_data['date'] = pd.to_datetime(full_data['date'])

In [4]:
grouped_by_data_count = full_data.groupby(['date', 'group'])[['cid', 'transactions', 'revenue']].agg({
    'cid': 'count',
    'transactions': 'sum',
    'revenue': 'sum'
    }
).reset_index().rename(
    columns={
        'cid': 'cid_count',
        'revenue': 'revenue_sum'
    }
)
grouped_by_data_count

,date,group,cid_count,transactions,revenue_sum
0,2020-11-06,A,2576,16,1577279
1,2020-11-06,B,2499,19,996645
2,2020-11-07,A,2704,23,1721886
3,2020-11-07,B,2804,19,555499
4,2020-11-08,A,3354,25,846051
5,2020-11-08,B,3291,24,3044842
6,2020-11-09,A,3234,18,708830
7,2020-11-09,B,3062,25,1285836
8,2020-11-10,A,3022,29,826984
9,2020-11-10,B,3072,30,2414726


In [5]:
grouped_by_data_count['cum_cid'] = grouped_by_data_count.groupby('group')['cid_count'].cumsum()
grouped_by_data_count['cum_trans'] = grouped_by_data_count.groupby('group')['transactions'].cumsum()
grouped_by_data_count['cum_rev'] = grouped_by_data_count.groupby('group')['revenue_sum'].cumsum()
grouped_by_data_count.head()

,date,group,cid_count,transactions,revenue_sum,cum_cid,cum_trans,cum_rev
0,2020-11-06,A,2576,16,1577279,2576,16,1577279
1,2020-11-06,B,2499,19,996645,2499,19,996645
2,2020-11-07,A,2704,23,1721886,5280,39,3299165
3,2020-11-07,B,2804,19,555499,5303,38,1552144
4,2020-11-08,A,3354,25,846051,8634,64,4145216


In [6]:
grouped_by_data_count['cum_mean_rev'] = grouped_by_data_count['cum_rev'] / grouped_by_data_count['cum_trans']
grouped_by_data_count['cum_conv'] = grouped_by_data_count['cum_trans'] / grouped_by_data_count['cum_cid'] * 100
grouped_by_data_count.head()

,date,group,cid_count,transactions,revenue_sum,cum_cid,cum_trans,cum_rev,cum_mean_rev,cum_conv
0,2020-11-06,A,2576,16,1577279,2576,16,1577279,98579.937500,0.621118
1,2020-11-06,B,2499,19,996645,2499,19,996645,52455.000000,0.760304
2,2020-11-07,A,2704,23,1721886,5280,39,3299165,84593.974359,0.738636
3,2020-11-07,B,2804,19,555499,5303,38,1552144,40845.894737,0.716576
4,2020-11-08,A,3354,25,846051,8634,64,4145216,64769.000000,0.741256


In [7]:
grouped_by_data_count.groupby('group')['cum_conv'].mean().round(2)

group
A    0.71
B    0.76
Name: cum_conv, dtype: float64

In [8]:
cum_conv_line = px.line(
    grouped_by_data_count,
    x='date',
    y='cum_conv',
    color='group',
    title='Графики кумулятивной конверсии для групп А и В'
)
cum_conv_line.show()

In [9]:
cum_mean_rev = px.line(
    grouped_by_data_count,
    x='date',
    y='cum_mean_rev',
    color='group',
    title='Графики кумулятивного среднего чека для групп А и В'
)
cum_mean_rev.show()

In [10]:
full_data.head()

,date,cid,transactions,revenue,group
0,2020-11-06,1.001648e+09,0,0,A
1,2020-11-06,1.001936e+09,0,0,A
2,2020-11-06,1.003214e+08,0,0,A
3,2020-11-06,1.004393e+09,0,0,A
4,2020-11-06,1.006382e+09,0,0,A


In [11]:
converted_piv = full_data.groupby('group')['transactions'].agg(
    ['sum', 'count']
)
converted_piv

,sum,count
group,,
A,441,61571
B,463,61383


In [12]:
from statsmodels.stats.proportion import proportions_ztest

In [15]:
h_0 = 'Нулевая гипотеза. Конверсия в группах А и В равна'
h_1 = 'Альтернативная гипотеза. Конверсия в группе А меньше, чем конверсия в группе В'
alpha = 0.1
ztest_result = proportions_ztest(
    count=converted_piv['sum'],
    nobs=converted_piv['count'],
    alternative='smaller'
)
print(f'p-value={ztest_result[1]}.', h_0) if ztest_result[1] > alpha else print(f'p-value={ztest_result[1]}.', h_1)

p-value=0.21753142255559005. Нулевая гипотеза. Конверсия в группах А и В равна


In [16]:
from scipy.stats import shapiro

In [18]:
h_0 = 'Нулевая гипотеза. Распределения в обеих группах являются нормальными.'
h_1 = 'Альтернативная гипотеза. Распределение в одной или обоих группах отличны от нормального'
alpha = 0.1

shapiro_a = shapiro(
    full_data[full_data['group'] == 'A']['revenue']
)
shapiro_b = shapiro(
    full_data[full_data['group'] == 'B']['revenue']
)
if shapiro_a.pvalue < alpha or shapiro_b.pvalue < alpha:
    print(h_1)
else:
    print(h_0)

Альтернативная гипотеза. Распределение в одной или обоих группах отличны от нормального


C:\Users\sadeyes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_morestats.py:1800: UserWarning:

p-value may not be accurate for N > 5000.



In [19]:
from scipy.stats import mannwhitneyu

In [21]:
full_data.head()

,date,cid,transactions,revenue,group
0,2020-11-06,1.001648e+09,0,0,A
1,2020-11-06,1.001936e+09,0,0,A
2,2020-11-06,1.003214e+08,0,0,A
3,2020-11-06,1.004393e+09,0,0,A
4,2020-11-06,1.006382e+09,0,0,A


In [25]:
daily_a = full_data[full_data['group'] == 'A'].groupby('date').agg(
    {
        'cid': 'count',
        'transactions': 'sum',
        'revenue': 'sum'
    }
).reset_index().rename(
    {
        'cid': 'total_users'
    }
)
daily_b = full_data[full_data['group'] == 'B'].groupby('date').agg(
    {
        'cid': 'count',
        'transactions': 'sum',
        'revenue': 'sum'
    }
).reset_index().rename(
    {
        'cid': 'total_users'
    }
)
daily_a['mean_rev'] = daily_a['revenue'] / daily_a['transactions']
daily_b['mean_rev'] = daily_b['revenue'] / daily_b['transactions']
display(
    daily_a.head(),
    daily_b.head()
)

,date,cid,transactions,revenue,mean_rev
0,2020-11-06,2576,16,1577279,98579.937500
1,2020-11-07,2704,23,1721886,74864.608696
2,2020-11-08,3354,25,846051,33842.040000
3,2020-11-09,3234,18,708830,39379.444444
4,2020-11-10,3022,29,826984,28516.689655


,date,cid,transactions,revenue,mean_rev
0,2020-11-06,2499,19,996645,52455.000000
1,2020-11-07,2804,19,555499,29236.789474
2,2020-11-08,3291,24,3044842,126868.416667
3,2020-11-09,3062,25,1285836,51433.440000
4,2020-11-10,3072,30,2414726,80490.866667


In [26]:
h_0 = 'Нулевая гипотеза. Средние чеки в группах А и В равны'
h_1 = 'Альтернативная гипотеза. Ежедневный средний чек в группе А меньше, чем в группе В'
alpha = 0.1

stest, pvalue = mannwhitneyu(
    x=daily_a['mean_rev'],
    y=daily_b['mean_rev'],
    alternative='less'
)
print(f'p-value={pvalue}.', h_0) if pvalue > alpha else print(f'p-value={pvalue}.', h_1)

p-value=0.08937067987846736. Альтернативная гипотеза. Ежедневный средний чек в группе А меньше, чем в группе В
